### Prepare stimuli in stereo with sync tone in the L channel and load into the SBC
To syncrhonize the recording systems, each stimulus file goes in stereo, the L channel has the stimulus, and the R channel has a pure tone (500-5Khz).
This is done here, with the help of the rigmq.util.stimprep module
It uses (or creates) a dictionary of {stim_file: tone_freq} which is stored as a .json file for offline processing.

In [1]:
import socket
import os
import sys
import logging
import warnings
import numpy as np
import glob

from rigmq.util import stimprep as sp

# setup the logger
logger = logging.getLogger()
handler = logging.StreamHandler()
formatter = logging.Formatter(
        '%(asctime)s %(name)-12s %(levelname)-8s %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.INFO)

# Check wich computer to decide where the things are mounted
comp_name=socket.gethostname()
logger.info('Computer: ' + comp_name)



2019-07-11 17:13:18,006 root         INFO     Computer: lookfar


In [2]:
exp_folder = os.path.abspath('/Users/zeke/experiment/birds')
bird = 'g18j18'
sess = 'acute_01'
stim_sf = 48000 # sampling frequency of the stimulus system

stim_folder = os.path.join(exp_folder, bird, 'SongData', sess)
glob.glob(os.path.join(stim_folder, '*.wav'))

['/Users/zeke/experiment/birds/g18j18/SongData/acute_01/20190705-f00287-calls.wav',
 '/Users/zeke/experiment/birds/g18j18/SongData/acute_01/20190705-f00201-bos-calls-undirected.wav',
 '/Users/zeke/experiment/birds/g18j18/SongData/acute_01/20190705-f00287-bos-calls-directed.wav',
 '/Users/zeke/experiment/birds/g18j18/SongData/acute_01/20190701-f03123-bos-undirected.wav',
 '/Users/zeke/experiment/birds/g18j18/SongData/acute_01/20190705-f00228-bos-directed.wav',
 '/Users/zeke/experiment/birds/g18j18/SongData/acute_01/20190705-f00254-bos-undirected.wav']

### Define stim_tags
There is a dictionary of {wav_file: tag_frequency} can be done by hand when there are few stimuli

In [3]:
stim_tags_dict = {'20190701-f03123-bos-undirected': 1000,
                  '20190705-f00201-bos-calls-undirected': 2000,
                  '20190705-f00228-bos-directed': 3000,
                 '20190705-f00287-bos-calls-directed': 4000,
                 '20190705-f00287-calls': 5000,
                 '20190705-f00254-bos-undirected': 2500}

stims_list = list(stim_tags_dict.keys())
sp.create_sbc_stim(stims_list, stim_folder, stim_sf, stim_tag_dict=stim_tags_dict)

2019-07-11 17:15:23,455 rigmq.util.stimprep INFO     Processing /Users/zeke/experiment/birds/g18j18/SongData/acute_01/20190701-f03123-bos-undirected.wav
2019-07-11 17:15:23,456 rigmq.util.stimprep INFO     tag_freq = 1000
2019-07-11 17:15:23,461 rigmq.util.stimprep INFO     Will resample from 78848 to 118272 sampes
2019-07-11 17:15:23,496 rigmq.util.stimprep INFO     Saved to /Users/zeke/experiment/birds/g18j18/SongData/acute_01/sbc_stim/20190701-f03123-bos-undirected_tag.wav
2019-07-11 17:15:23,497 rigmq.util.stimprep INFO     Processing /Users/zeke/experiment/birds/g18j18/SongData/acute_01/20190705-f00201-bos-calls-undirected.wav
2019-07-11 17:15:23,498 rigmq.util.stimprep INFO     tag_freq = 2000
2019-07-11 17:15:23,506 rigmq.util.stimprep INFO     Will resample from 73288 to 109932 sampes
2019-07-11 17:15:24,194 rigmq.util.stimprep INFO     Saved to /Users/zeke/experiment/birds/g18j18/SongData/acute_01/sbc_stim/20190705-f00201-bos-calls-undirected_tag.wav
2019-07-11 17:15:24,195 ri

In [4]:
files_list = list(stim_tags_dict.keys())
files_list

['20190701-f03123-bos-undirected',
 '20190705-f00201-bos-calls-undirected',
 '20190705-f00228-bos-directed',
 '20190705-f00287-bos-calls-directed',
 '20190705-f00287-calls',
 '20190705-f00254-bos-undirected']

### Copy the stimulus files to the raspberry pi

In [6]:
pi_address = 'raspberrypi.local'
pi_dest = 'pi@{}:~/stim_files'.format(pi_address)
wav_source = os.path.join(stim_folder, 'sbc_stim', '*.wav')

In [7]:
!rsync -v {wav_source} {pi_dest}

20190701-f03123-bos-undirected_tag.wav
20190705-f00201-bos-calls-undirected_tag.wav
20190705-f00228-bos-directed_tag.wav
20190705-f00254-bos-undirected_tag.wav
20190705-f00287-bos-calls-directed_tag.wav
20190705-f00287-calls_tag.wav

sent 2,711,478 bytes  received 130 bytes  5,423,216.00 bytes/sec
total size is 2,710,308  speedup is 1.00


In [8]:
!ping raspberrypi.local

PING raspberrypi.local (192.168.1.50): 56 data bytes
64 bytes from 192.168.1.50: icmp_seq=0 ttl=64 time=1.622 ms
64 bytes from 192.168.1.50: icmp_seq=1 ttl=64 time=11.288 ms
64 bytes from 192.168.1.50: icmp_seq=2 ttl=64 time=7.753 ms
64 bytes from 192.168.1.50: icmp_seq=3 ttl=64 time=7.373 ms
64 bytes from 192.168.1.50: icmp_seq=4 ttl=64 time=7.555 ms
64 bytes from 192.168.1.50: icmp_seq=5 ttl=64 time=1.525 ms
64 bytes from 192.168.1.50: icmp_seq=6 ttl=64 time=17.185 ms
64 bytes from 192.168.1.50: icmp_seq=7 ttl=64 time=2.003 ms
64 bytes from 192.168.1.50: icmp_seq=8 ttl=64 time=27.807 ms
64 bytes from 192.168.1.50: icmp_seq=9 ttl=64 time=1.448 ms
64 bytes from 192.168.1.50: icmp_seq=10 ttl=64 time=7.860 ms
^C

--- raspberrypi.local ping statistics ---
11 packets transmitted, 11 packets received, 0.0% packet loss
round-trip min/avg/max/stddev = 1.448/8.493/27.807/7.669 ms
